To start I proceed I import pandas and the csv document to start working, to read it I use the "Cp1252" encoding


In [1]:
import pandas as pd
import numpy as np
import re
#abro el documento
att = pd.read_csv("INPUTS/attacks.csv", encoding = "Cp1252")

I explore rows and columns in order to look for duplicates, to dismiss useless columns and also rows that have inner inconsistencies

In [2]:
#att.head() #I do a first survey to explore data
#display(att.head())

In [3]:
#att.columns


In [4]:
att.shape

(25723, 24)

In [5]:
 #I explore null values in order to decide whether delete rows, columns
#change null NaN values for other values
#att.isnull().sum().sort_values(ascending=True)

In [6]:
#att[["Year", "Type", "Injury", "Country", "Area", "Fatal (Y/N)", "Age", "Time"]]

In [7]:
#att[["Case Number.1", "Case Number.2", "href", "pdf", "href formula"]]

In [8]:
#att[["Fatal (Y/N)", "Location", "Activity", "Species ", "Time"]].head(60)

In [9]:
#att[["Unnamed: 22", "Unnamed: 23"]] #Estas dos las eliminaré

In [10]:
att_s = att.drop(columns= ["Unnamed: 22", "Unnamed: 23","pdf","Name", "Sex ", "Investigator or Source", "pdf", "href formula", "href"])     

In [11]:
for col, numnulls in att_s.isnull().sum().items():
    if numnulls > 0:
        print(f"{col} {numnulls}")

Case Number 17021
Date 19421
Year 19423
Type 19425
Country 19471
Area 19876
Location 19961
Activity 19965
Age 22252
Injury 19449
Fatal (Y/N) 19960
Time 22775
Species  22259
Case Number.1 19421
Case Number.2 19421
original order 19414


In [12]:
#att_s.columns

I delete all the rows with all NaN values in all the columns.
Following this, I rename the columns that I might use and that have ambiguous names 

In [13]:
att_s.rename(columns={'Case Number':"CaseNumber",'Fatal (Y/N)': 'Fatal', 'Species ': 'Species','Case Number.1': 'Case1', 'Case Number.2': 'Case2'}, inplace=True)


In [14]:
att_clean = att_s.dropna(how = "all")
att_clean.shape

(8703, 16)

I go forth and back exploring the data to decide which columns should I keep. I select some, look at the resulting dataFrame.  

In [15]:
#att_clean.head(60)


In [16]:

att_clean2 = att_clean.drop(columns= ["CaseNumber", "Case2", "Year", "Location", "Time", "Case1", "Species"])


But still there are some rows left with lots of NaN values, so I decide to remove them manually as they do not contain information of interest for my study


In [17]:
att_clean2 = att_clean2.dropna(how = "all")
att_clean2.tail()


,Date,Type,Country,Area,Activity,Age,Injury,Fatal,original order
6304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6306.0
6305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6307.0
6306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6308.0
6307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6309.0
6308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6310.0


In [18]:
study_att = att_clean2.drop([6302, 6303, 6304, 6305, 6306, 6307, 6308], axis = 0) 

In [19]:
#Over the data that I plan to use to test my hypothesis, I explore, again null values.
study_att.isnull().sum().sort_values(ascending=False).head(10)


Age               2831
Activity           544
Fatal              539
Area               455
Country             50
Injury              28
Type                 4
original order       0
Date                 0
dtype: int64

In [20]:
#study_att.columns

In [21]:
#Now I rename the content of cells in order to 
study_att[["Fatal", "Injury"]].head()


,Fatal,Injury
0,N,"No injury to occupant, outrigger canoe and pad..."
1,N,Minor injury to left thigh
2,N,Injury to left lower leg from surfboard skeg
3,N,Minor injury to lower leg
4,N,Lacerations to leg & hand shark PROVOKED INCIDENT


In [22]:
study_att.Injury.value_counts().head()


FATAL          802
Survived        97
Foot bitten     87
No injury       82
Leg bitten      72
Name: Injury, dtype: int64

In [23]:
study_att.Fatal.value_counts()


N          4293
Y          1388
UNKNOWN      71
 N            7
M             1
2017          1
y             1
N             1
Name: Fatal, dtype: int64

In [24]:
pd.set_option('display.max_rows', None)
#display(study_att)



In [25]:
#pd.set_option('display.max_rows', None)
#study_att.Injury.value_counts()


In [26]:
pd.set_option('display.max_rows', None)
study_att[["Fatal", "Injury"]].head()


,Fatal,Injury
0,N,"No injury to occupant, outrigger canoe and pad..."
1,N,Minor injury to left thigh
2,N,Injury to left lower leg from surfboard skeg
3,N,Minor injury to lower leg
4,N,Lacerations to leg & hand shark PROVOKED INCIDENT


In [27]:
#To check the relationship between NaN in Injury and the content of Fatal. 
#Given that in both cases there is no data, these rows can be deleted.

study_att[study_att.Injury.isnull()][["Injury","Fatal"]]


,Injury,Fatal
1270,NaN,UNKNOWN
1366,NaN,NaN
2250,NaN,UNKNOWN
2267,NaN,UNKNOWN
2648,NaN,UNKNOWN
3038,NaN,UNKNOWN
3207,NaN,NaN
3298,NaN,UNKNOWN
3434,NaN,UNKNOWN
3457,NaN,NaN


In [28]:
study_att.dropna(subset=['Injury'], inplace=True)

In [29]:
study_att[study_att.Fatal.isnull()][["Injury","Fatal", "Age"]]


,Injury,Fatal,Age
27,"Multiple severe injuries to arms and leg, leg ...",NaN,34
76,FATAL,NaN,31
79,"Lacerations to right ankle, foot & toe",NaN,33
84,"FATAL, but death was probably due to drowning",NaN,25
98,Lacerations to left foot,NaN,11
107,Abrasions and cuts to sole of foot,NaN,10
115,Foot injured,NaN,8
124,"Skull found in shark, a probable drowning & sc...",NaN,NaN
125,"Arm injured by hook, not by a shark",NaN,NaN
145,"Laceration to thigh, likely caused by surfboar...",NaN,18


In [30]:
study_att["Fatal"].fillna("N", inplace=True)
study_att["Fatal"].isnull().sum()


0

In [31]:
#Given that null values in Injury correspond to doubtfull attacks of sharks, I decide to drop these rows in order to avoid untruthful information
study_att["Injury"].fillna("N", inplace=True)
study_att["Injury"].isnull().sum()


0

In [32]:
study_att["Activity"].isnull().sum()



532

In [33]:
study_att.Activity.value_counts()
study_att[study_att.Activity.isnull()][["Injury","Fatal", "Activity"]]
#Given that NaN Activity mainly means that the activity is unknown, I decide to change null values into "Unknown"
study_att["Activity"] = study_att["Activity"].fillna("Unknown")

In [34]:
study_att["Activity"].isnull().sum()



0

In [35]:
study_att.loc[study_att["Fatal"].str.startswith(" N")] = "N"
study_att.loc[study_att["Fatal"].str.startswith("N ")] = "N"
study_att.loc[study_att["Fatal"].str.startswith("M")] = "N"
study_att.loc[study_att["Fatal"].str.startswith("y")] = "Y"
study_att.loc[study_att["Fatal"].str.startswith("2017")] = "N"

study_att.Fatal.value_counts()


N          4835
Y          1388
UNKNOWN      51
Name: Fatal, dtype: int64

In [40]:
study_att = study_att.drop(columns= ["original order", "Date", "Age", "Area", "Type"])

In [41]:
study_att

,Country,Activity,Injury,Fatal
0,USA,Paddling,"No injury to occupant, outrigger canoe and pad...",N
1,USA,Standing,Minor injury to left thigh,N
2,USA,Surfing,Injury to left lower leg from surfboard skeg,N
3,AUSTRALIA,Surfing,Minor injury to lower leg,N
4,MEXICO,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N
5,AUSTRALIA,Kite surfing,"No injury, board bitten",N
6,BRAZIL,Swimming,FATAL,Y
7,USA,Fishing,Minor injury to foot. PROVOKED INCIDENT,N
8,USA,Walking,Lower left leg bitten,N
9,USA,Standing,Minor injury to foot,N


In [42]:
study_att.shape

(6274, 4)

In [44]:
study_att.Injury.value_counts()


FATAL                                                                                                                                                                                                                                          801
Survived                                                                                                                                                                                                                                        96
Foot bitten                                                                                                                                                                                                                                     87
No injury                                                                                                                                                                                                                                       82
Leg bitten                  

In [46]:

def InjuryType(x):
    if "fatal" in x.lower:
        return "Fatal"
    elif "No harm"or "No Injury" in x.lower:
        return "No harm"
    elif "Minor" in x.lower:
        return "Minor"
    elif "Laceration" or "lacerate" in x.lower:
        return "Lacerations"
    elif "Bitten" or "bitten" or "bites" in x.lower:
        return "Bitten"
    else: 
        return x.lower

study_att["Injury_type"] = study_att['Injury'].map(InjuryType)



TypeError: argument of type 'builtin_function_or_method' is not iterable

In [81]:
#study_att["Injury_level"] = np.where(study_att.Injury.str.startswith("Fata"), "Fatal", "Minor")
#["Injury"][study_att.Injury.str.startswith("Fatal")] = "Fatal"
#study_att["Injury"][study_att.Injury.str.startswith("Minor")] = "Minor"

#study_att["Injury_level"] = study_att.Injury
#study_att.loc[study_att["Injury"].str.startswith("Fatal"), "Injury"] = "Fatal"
#study_att.loc[study_att["Injury"].str.startswith("fatal"), "Injury"] = "Fatal"
#study_att.loc[study_att["Injury"].str.startswith("FATAL"), "Injury"] = "Fatal"
#study_att.loc[study_att["Injury"].str.startswith("Death"), "Injury"] = "Fatal"

#study_att.loc[study_att["Injury"].str.startswith("Minor"), "Injury"] = "Minor"
#study_att.loc[study_att["Injury"].str.startswith("Severe"), "Injury"] = "Severe"
#study_att.loc[study_att["Injury"].str.startswith("Leg"), "Injury"] = "Middle"
#study_att.loc[study_att["Injury"].str.startswith("leg"), "Injury"] = "Middle"
#study_att.loc[study_att["Injury"].str.startswith("left"), "Injury"] = "Middle"
#study_att.loc[study_att["Injury"].str.startswith("foot"), "Injury"] = "Middle"
#study_att.loc[study_att["Injury"].str.startswith("right"), "Injury"] = "Middle"
#study_att.loc[study_att["Injury"].str.startswith("Right"), "Injury"] = "Middle"
#study_att.loc[study_att["Injury"].str.startswith("Right"), "Injury"] = "Middle"

#study_att.loc[study_att["Injury"].str.startswith("No"), "Injury"] = "No harm"
study_att["Injury"] = study_att.apply(unify, axis=0)

#MEJOR QUE ESTO, DEBERÍA HACER UNA FUNCIÓN

study_att.loc[study_att["Injury"].str.startswith("Fata")] = "Fatal"
study_att.loc[study_att["Injury"].str.startswith("Minor")] = "Minor Injury"
study_att.loc[study_att["Injury"].str.startswith("Severe")] = "Severe"

NameError: name 'study_att' is not defined